In [1]:
#!pip install sranodec

In [15]:
import pandas as pd
#from preprocess import *
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import datetime

In [2]:
df = pd.read_csv("./data/merged_annuale_meteo.csv", delimiter=";")
#df.set_index("timestamp", inplace=True)
df['timestamp'] = pd.to_datetime(df['timestamp'], yearfirst=True)
timestamp = df['timestamp'].copy()

df.drop(columns="timestamp", inplace=True)
for column in df:
    if "Total" in column:
    # rende non cumulativo l'andamento di total energy
        df[column] = df[column].diff().fillna(0)
#df = df.sort_values("timestamp")


In [3]:
target_features = [
    "Cont_TotalEnergy(kWh)",
    "timestamp",
    'temperature_2m (°C)',
    'relativehumidity_2m (%)',
    'rain (mm)',
    'temperature_2m',
    'dewpoint_2m'
    'cloudcover (%)',
    'soil_moisture_7_to_28cm (m³/m³)',
    'soil_temperature_7_to_28cm (°C)'
]

exclude_pcov_features= [
    "Failure",
    "Frequency",
    "Status",
    "Others",
    "Code",
    "name",
    "deviceid",
    "serial",
    "SB0",
    "NV10P",
    "NA16",
    "Phase",
    "INV01_TotalEnergy",
    "INV02_TotalEnergy",
    "INV03_TotalEnergy",
    "INV01_CurrentDC",
    "INV01_CurrentAC",
    "INV01",
   # "INV02_PowerAC",
    #"INV03_PowerAC",
    #"INV02_PowerDC",
    #"INV03_PowerDC",
    #"INV02_CurrentAC",
    #"INV02_CurrentDC",
    #"INV03_CurrentDC",
    #"INV03_CurrentAC",
    "state",
    #'soil_moisture_7_to_28cm (m³/m³)',
]

In [4]:
f_df = pd.DataFrame()

columns = []


for column in df:
  #if not any(key in column for key in exclude_pcov_features + future_cov_features):
  if not any(key in column for key in exclude_pcov_features) or any(key in column for key in target_features):
    f_df[column] =  df[column].copy()
    columns.append(column)
  '''
  if not any(key in column for key in target_features):
    df.drop(columns = [column], axis = 1, inplace = True)
  '''
#columns.pop(0)
f_df.drop(columns=["Cont_TotalEnergy_QuotControl(Wh)", "Cont_TotalEnergyImported(kWh)"], inplace=True)



In [5]:
print(timestamp)

0        2022-02-02 00:05:00
1        2022-02-02 00:10:00
2        2022-02-02 00:15:00
3        2022-02-02 00:20:00
4        2022-02-02 00:25:00
                 ...        
143425   2023-06-16 17:45:00
143426   2023-06-16 17:50:00
143427   2023-06-16 17:55:00
143428   2023-06-16 18:00:00
143429   2023-06-16 18:05:00
Name: timestamp, Length: 143430, dtype: datetime64[ns]


In [6]:
f_df["timestamp"] = timestamp

data = f_df[34232:].copy(deep=True)
data = data.fillna(method="ffill", axis=1)

#timestamp = data['timestamp'].copy()

#data['timestamp'] = pd.to_datetime(data['timestamp'], yearfirst=True)
#data.set_index("timestamp", inplace=True)



In [7]:

#data['timestamp'] = pd.to_datetime(data['timestamp'], yearfirst=True)

data.set_index("timestamp", inplace=True)
temp_df = pd.DataFrame()
    

for col in data:
    data[col]= data[col].astype(np.float32)
    if all(key in col for key in "Temperature(C)"):
        temp_df[col] = data[col].copy(deep=True)


data = data.resample('15T').sum()

temp_df= temp_df.resample('15T').mean()
   
                

timestamp = data.index

scaler = MinMaxScaler()

# Fit and transform the DataFrame using the scaler
#scaled_data = scaler.fit_transform(data)



In [34]:
data

,INV03_CurrentDC(A),INV03_CurrentAC(A),INV03_PowerAC(kW),INV03_PowerDC(kW),INV03_InternalTemperature(C),INV03_HeatSinkTemperature(C),INV03_VoltageDC(V),INV03_VoltageAC(V),INV02_CurrentDC(A),INV02_CurrentAC(A),...,day_cos,month_sin,month_cos,temperature_2m (°C),relativehumidity_2m (%),dewpoint_2m (°C),rain (mm),cloudcover (%),soil_temperature_7_to_28cm (°C),soil_moisture_7_to_28cm (m³/m³)
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-06-01 06:30:00,28.020000,25.629999,10.910000,12.950000,26.260000,35.270000,463.410004,253.320007,26.780001,27.330000,...,0.979530,1.224647e-16,-1.0,-1.000000,86.0,17.700001,0.0,0.0,22.700001,0.201
2022-06-01 06:45:00,105.139999,105.990005,45.709999,52.290001,79.639999,111.820000,1485.829956,760.160034,101.759995,111.850006,...,2.938590,3.673941e-16,-3.0,-3.000000,258.0,53.100002,0.0,0.0,68.100006,0.603
2022-06-01 07:00:00,144.580002,168.779999,73.139999,81.059998,81.489998,126.289993,1679.169922,758.690002,146.540009,172.089996,...,2.938590,3.673941e-16,-3.0,19.100000,240.0,52.199997,0.0,0.0,67.500000,0.603
2022-06-01 07:15:00,202.239990,244.290009,107.000000,116.479996,83.309998,147.259995,1727.770020,763.660034,207.029999,248.940002,...,2.938590,3.673941e-16,-3.0,-3.000000,240.0,52.199997,0.0,0.0,67.500000,0.603
2022-06-01 07:30:00,271.540009,332.289978,146.380005,157.699997,85.360001,168.450012,1742.510010,767.089966,276.700012,338.750000,...,2.938590,3.673941e-16,-3.0,-3.000000,240.0,52.199997,0.0,0.0,67.500000,0.603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-16 17:00:00,330.390015,378.760010,173.259995,185.380005,87.290001,101.540001,1683.979980,797.709961,332.059998,385.260010,...,-2.984608,3.673941e-16,-3.0,19.600000,186.0,42.300003,0.0,135.0,62.699997,1.116
2023-06-16 17:15:00,316.979980,379.229980,170.679993,182.929993,88.330002,100.970001,1732.439941,784.900024,320.100006,383.940002,...,-2.984608,3.673941e-16,-3.0,-3.000000,186.0,42.300003,0.0,135.0,62.699997,1.116
2023-06-16 17:30:00,348.169983,415.059998,186.119995,198.869995,87.709999,102.239998,1712.679932,780.739990,346.119995,420.799988,...,-2.984608,3.673941e-16,-3.0,-3.000000,186.0,42.300003,0.0,135.0,62.699997,1.116


In [37]:
sunsetrise = pd.read_csv("./data/sunriseset.csv", header=2, sep=',', index_col=False, parse_dates=[
    'time', 'sunrise (iso8601)', 'sunset (iso8601)'])
sunsetrise.index = sunsetrise['time']
sunsetrise.drop(columns=['time'], inplace=True)
sunsetrise.columns = ['sunrise', 'sunset']
sunsetrise = sunsetrise.loc[data.index[0]:data.index[-1]]

In [38]:
sunsetrise

,sunrise,sunset
time,,
2022-06-02,2022-06-02 05:21:00,2022-06-02 20:18:00
2022-06-03,2022-06-03 05:21:00,2022-06-03 20:19:00
2022-06-04,2022-06-04 05:20:00,2022-06-04 20:20:00
2022-06-05,2022-06-05 05:20:00,2022-06-05 20:20:00
2022-06-06,2022-06-06 05:20:00,2022-06-06 20:21:00
...,...,...
2023-06-12,2023-06-12 05:19:00,2023-06-12 20:24:00
2023-06-13,2023-06-13 05:18:00,2023-06-13 20:25:00
2023-06-14,2023-06-14 05:18:00,2023-06-14 20:25:00


In [41]:
def custom_isday(sunrise, sunset):
    daydf = pd.DataFrame(
        0,
        index=data[
            ((data.index >= pd.to_datetime(sunrise.date())) * 
             (data.index < pd.to_datetime(sunrise.date() + datetime.timedelta(days=1))))
        ].index,
        columns=['isday']
    )
    
    #print(daydf)
    
    # print(tmp_df.shape)
    
    start = nearest(daydf.index, sunrise)
    end = nearest(daydf.index,sunset)
    
    # print(start, "-", end)

    daydf[((daydf.index >= start) * (daydf.index <= end))] = 1
    
    return daydf

In [42]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

In [43]:
customisday = pd.DataFrame()

for index, (sunrise, sunset) in sunsetrise.iterrows():
    #print(f"index:{index}, sunrise:{sunrise}, sunset:{sunset}")
    test = custom_isday(sunrise, sunset)
    customisday = pd.concat([customisday, test])

In [44]:
customisday

,isday
timestamp,
2022-06-02 00:00:00,0
2022-06-02 00:15:00,0
2022-06-02 00:30:00,0
2022-06-02 00:45:00,0
2022-06-02 01:00:00,0
...,...
2023-06-16 17:00:00,1
2023-06-16 17:15:00,1
2023-06-16 17:30:00,1


In [45]:
data = pd.concat([data, customisday], axis=1)

In [48]:
#data = pd.DataFrame(scaled_data, columns=data.columns)
data.reset_index(drop=True,inplace=True)
#data['timestamp'] = pd.to_datetime(data['timestamp'], yearfirst=True)
print(data)

       INV03_CurrentDC(A)  INV03_CurrentAC(A)  INV03_PowerAC(kW)  \
0               28.020000           25.629999          10.910000   
1              105.139999          105.990005          45.709999   
2              144.580002          168.779999          73.139999   
3              202.239990          244.290009         107.000000   
4              271.540009          332.289978         146.380005   
...                   ...                 ...                ...   
36522          330.390015          378.760010         173.259995   
36523          316.979980          379.229980         170.679993   
36524          348.169983          415.059998         186.119995   
36525          320.609985          369.750000         165.759995   
36526          144.739990          159.480011          71.130005   

       INV03_PowerDC(kW)  INV03_InternalTemperature(C)  \
0              12.950000                     26.260000   
1              52.290001                     79.639999   
2        

In [49]:
temp_df.reset_index(drop=True,inplace=True)

for col in temp_df:
    data[col]=temp_df[col].copy()
print(data)

       INV03_CurrentDC(A)  INV03_CurrentAC(A)  INV03_PowerAC(kW)  \
0               28.020000           25.629999          10.910000   
1              105.139999          105.990005          45.709999   
2              144.580002          168.779999          73.139999   
3              202.239990          244.290009         107.000000   
4              271.540009          332.289978         146.380005   
...                   ...                 ...                ...   
36522          330.390015          378.760010         173.259995   
36523          316.979980          379.229980         170.679993   
36524          348.169983          415.059998         186.119995   
36525          320.609985          369.750000         165.759995   
36526          144.739990          159.480011          71.130005   

       INV03_PowerDC(kW)  INV03_InternalTemperature(C)  \
0              12.950000                     26.260000   
1              52.290001                     26.546667   
2        

In [50]:
#data.drop(columns="timestamp", inplace=True)
#data.drop(columns="index", inplace=True)
#data.reset_index(drop=True,inplace=True)

data['timestamp'] = timestamp.values
print(data)

       INV03_CurrentDC(A)  INV03_CurrentAC(A)  INV03_PowerAC(kW)  \
0               28.020000           25.629999          10.910000   
1              105.139999          105.990005          45.709999   
2              144.580002          168.779999          73.139999   
3              202.239990          244.290009         107.000000   
4              271.540009          332.289978         146.380005   
...                   ...                 ...                ...   
36522          330.390015          378.760010         173.259995   
36523          316.979980          379.229980         170.679993   
36524          348.169983          415.059998         186.119995   
36525          320.609985          369.750000         165.759995   
36526          144.739990          159.480011          71.130005   

       INV03_PowerDC(kW)  INV03_InternalTemperature(C)  \
0              12.950000                     26.260000   
1              52.290001                     26.546667   
2        

In [51]:
data.to_csv("./data/merged_annuale_meteo_sunrise_filtrato15.csv", sep=";", index=False)
